In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext

conf = SparkConf().setMaster("local").setAppName("Mi programa")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

dfspark = sqlContext.read.format("csv").option("header", "true").option("inferSchema","true").load("C:/Users/Gabo0/Desktop/Jupyter Python/Ejercicios Linkedin/base_datos_2008.csv")
dfspark = dfspark.sample(fraction = 0.001, withReplacement=False)
dfspark = dfspark.withColumn("ArrDelay",dfspark["ArrDelay"].cast("integer"))

df2 = dfspark.na.drop(subset=['ArrDelay','DepDelay','Distance'])
df2 = df2.filter("ArrDelay is not NULL")
df2 = df2.dropDuplicates()

In [2]:
A = sc.parallelize(df2.select("Origin").rdd.collect())

In [3]:
A.persist()

ParallelCollectionRDD[19] at readRDDFromFile at PythonRDD.scala:274

In [4]:
mapfunction = A.map(lambda x: (x,1))

In [5]:
def fun(x):
    if x[0] in ["SEA","ATL","HOU"]:
        return((x,2))
    elif x[0] =="DEN":
        return((x,3))
    else:
        return((x,1))

In [6]:
mapfunction2=A.map(fun)

In [7]:
reducefunction = mapfunction.reduceByKey(lambda x,y: x+y)

In [8]:
reducefunction2 = mapfunction2.reduceByKey(lambda x,y: x+y)

In [9]:
reducefunction.collect()

[(Row(Origin='SMF'), 39),
 (Row(Origin='PDX'), 64),
 (Row(Origin='ATL'), 408),
 (Row(Origin='MIA'), 84),
 (Row(Origin='ORD'), 324),
 (Row(Origin='PHX'), 179),
 (Row(Origin='CLE'), 66),
 (Row(Origin='BDL'), 21),
 (Row(Origin='MDW'), 81),
 (Row(Origin='HOU'), 63),
 (Row(Origin='CLT'), 127),
 (Row(Origin='IAD'), 74),
 (Row(Origin='BOS'), 136),
 (Row(Origin='GRR'), 15),
 (Row(Origin='ONT'), 29),
 (Row(Origin='OKC'), 21),
 (Row(Origin='LAS'), 173),
 (Row(Origin='IAH'), 178),
 (Row(Origin='DFW'), 245),
 (Row(Origin='MSP'), 103),
 (Row(Origin='LAX'), 203),
 (Row(Origin='BUF'), 22),
 (Row(Origin='SFO'), 141),
 (Row(Origin='DEN'), 240),
 (Row(Origin='DTW'), 146),
 (Row(Origin='SYR'), 16),
 (Row(Origin='LYH'), 1),
 (Row(Origin='MEM'), 82),
 (Row(Origin='CVG'), 104),
 (Row(Origin='AEX'), 4),
 (Row(Origin='LGA'), 106),
 (Row(Origin='VPS'), 6),
 (Row(Origin='RDU'), 63),
 (Row(Origin='FLL'), 78),
 (Row(Origin='BUR'), 39),
 (Row(Origin='BMI'), 5),
 (Row(Origin='TUS'), 31),
 (Row(Origin='MHT'), 15),
 

In [10]:
reducefunction.sortByKey().take(10)

[(Row(Origin='ABE'), 4),
 (Row(Origin='ABI'), 2),
 (Row(Origin='ABQ'), 33),
 (Row(Origin='ACK'), 1),
 (Row(Origin='ACV'), 6),
 (Row(Origin='ADQ'), 1),
 (Row(Origin='AEX'), 4),
 (Row(Origin='AGS'), 2),
 (Row(Origin='ALB'), 9),
 (Row(Origin='AMA'), 11)]

In [13]:
reducefunction2.sortBy(lambda x: x[1], ascending=False).take(10)

[(Row(Origin='ATL'), 816),
 (Row(Origin='DEN'), 720),
 (Row(Origin='ORD'), 324),
 (Row(Origin='DFW'), 245),
 (Row(Origin='LAX'), 203),
 (Row(Origin='PHX'), 179),
 (Row(Origin='IAH'), 178),
 (Row(Origin='SEA'), 174),
 (Row(Origin='LAS'), 173),
 (Row(Origin='DTW'), 146)]